In [1]:
""" MonkeyPie
    
    FIND FUZZY DUPLICATES FOR SONGS ACROSS ANY CODEC
    - include check better quality, including probable glitch/skip analysis
    
    - matching inc. levenshtein fuzzy-compare of the "artist + songtitle"
        and of audio profiles
        
"""

""" PROCESS
    
    - run and create analysis-sets for all possible duplicates
        - is pkl with song ID, path [ and dbItem object? ]
    - sequentially place each analysis-set in now playing, 
        - first check now playing is empty (on start, not each dupset) and offer store as playlist
    - prompt user to play through all in the set and either:
        - remove undesired duplicates from now playing, and enter "keep remaining" into [console? popup? ]
            - marks remaining set as "non-duplicate" in a pkl db for future reference
            - move excluded out of library and filedir, to separate subdir in "dups_removed"
        - go next or quit
    
"""

' PROCESS\n    \n    - run and create analysis-sets for all possible duplicates\n        - is pkl with song ID, path [ and dbItem object? ]\n    - sequentially place each analysis-set in now playing, \n        - first check now playing is empty (on start, not each dupset) and offer store as playlist\n    - prompt user to play through all in the set and either:\n        - remove undesired duplicates from now playing, and enter "keep remaining" into [console? popup? ]\n            - marks remaining set as "non-duplicate" in a pkl db for future reference\n            - move excluded out of library and filedir, to separate subdir in "dups_removed"\n        - go next or quit\n    \n'

In [2]:
import win32com.client
# import sys
# import time
# import Levenshtein
# import codecs

In [3]:
def get_songsDB(
    dispatch = 'SongsDB.SDBApplication',
    shutdown = False
    ):
        SDB = win32com.client.Dispatch( dispatch )  # check how interacts w. mmnk
        SDB.ShutdownAfterDisconnect = shutdown
        return SDB

def getResults_pathLike ( W32_SongsDB, matchStr ):
    results = []
    seltracks = W32_SongsDB.Database.QuerySongs( 
        f"Songs.SongPath LIKE '%{ matchStr }'")
        
    while not seltracks.EOF:
        resultItem = seltracks.Item
        results.append(resultItem)
        seltracks.Next()
        
    print( f"QUERY [ {matchStr} ] RETURNED [ { len(results)} ] RESULTS " )
    W32_SongsDB = None
    return results

In [135]:
def main():
    
    SongsDB = get_songsDB()
    dbItems_flac = getResults_pathLike ( SongsDB, ".flac" )
    
    return dbItems_flac

# if __name__ == '__main__':
flacs = main()

# takes ~8 sec to return ~8000 results from ~44000 files

QUERY [ .flac ] RETURNED [ 8149 ] RESULTS 


In [7]:
# ANY song, path is like "G:\E MUSIC"

SongsDB = get_songsDB()
dbItems_flac = getResults_pathLike ( SongsDB, ".flac" )
len(dbItems_flac)

QUERY [ .flac ] RETURNED [ 8149 ] RESULTS 


8149

In [8]:
SongsDB = get_songsDB()
dbItems_mp3 = getResults_pathLike ( SongsDB, ".mp3" )
len(dbItems_mp3)

QUERY [ .mp3 ] RETURNED [ 35944 ] RESULTS 


35944

In [10]:
dbItems_wav = getResults_pathLike ( SongsDB, ".wav" )
len(dbItems_wav)

QUERY [ .wav ] RETURNED [ 10 ] RESULTS 


10

In [11]:
len(dbItems_flac) + len(dbItems_mp3) + len(dbItems_wav)

44103

In [12]:
monkey_db_items = dbItems_flac + dbItems_mp3 + dbItems_wav

In [13]:
# # Get track object attributes
# dirStrs = sorted( [ attrStr for attrStr in flacs[0].__dir__() ] )
# for dirStr in dirStrs: print( dirStr )

In [15]:
attribsSelect = [
    'ID',
    'Title',
    'ArtistName',
    'SongLengthString',
    'AlbumName',
    'AlbumArtistName',
    'MusicComposer',
    'Encoder',
    'Bitrate',
    'SampleRate',
    'Channels',
    'FileLength',
    'Date',
    'LastPlayed',
    'DateAdded',
    'DateDBModified',
    'FileModified',
    'DiscNumber',
    'TrackOrder',
    'Genre',
    'Path',
    'PlayCounter',
    'SkipCount',
    'SongID',
    'SongLength',
]

In [16]:
def getAttrValPair( obj, attString, toConsole=False ):
    
    value = eval( f"obj.{attString}" )
    
    if toConsole:
        valStr = str(value)[:60]
        if len(valStr) > 59: valStr += "..."
        padd = max([ len(attrStr) for attrStr in attribsSelect ]) + 2
        print( f"{attString}:{ ' ' * (padd-len(attString)) }{valStr}" )
        
    else: return ( attString, value )

In [17]:
def fileLengthConvert( byteSize, mb = True ):
    # disagrees (less than by ~1mb) win filesys "size" field
    outSize = byteSize / 1024
    if mb: return outSize / 1024
    return outSize

In [ ]:

size = fileLengthConvert( flacs[0].FileLength )
size

In [112]:
# output selected track attributes
for attrStr in attribsSelect: getAttrValPair( flacs[0], attrStr, toConsole=True )

ID:                33508
Title:             Gymnopedie for piano No. 1
ArtistName:        Tamara-Anna Cislowski
SongLengthString:  3:25
AlbumName:         The Persian Hours
AlbumArtistName:   Tamara-Anna Cislowski
MusicComposer:     Erik Satie
Encoder:           FLAC 1.3.3
Bitrate:           1416344
SampleRate:        44100
Channels:          2
FileLength:        36307998
Date:              2008-01-01 00:00:00+00:00
LastPlayed:        1899-12-30 00:00:00+00:00
DateAdded:         2021-08-27 11:33:08.384000+00:00
DateDBModified:    2021-08-27 13:09:57.089000+00:00
FileModified:      2021-08-12 01:01:45.704000+00:00
DiscNumber:        1
TrackOrder:        1
Genre:             Classical
Path:              G:\E MUSIC\flac\Tamara-Anna Cislowski\The Persian Hours\000 ...
PlayCounter:       0
SkipCount:         0
SongID:            33508
SongLength:        205000


In [45]:
mm_lib = {}

for tr_obj in monkey_db_items:
    dct = { att : eval( f"tr_obj.{att}" ) for att in attribsSelect }
    mm_lib.update( { dct["ID"] : dct } )

len( mm_lib.keys() )

44103

In [49]:
import pandas as pd

In [50]:

df = pd.DataFrame( mm_lib )




In [51]:
df

,33508,33509,33510,33511,33512,33513,33514,33515,33516,33517,33518,33519,33520,33521,33522,33523,33524,33525,33526,33527,33528,33529,33530,33531,33532,33533,33534,33535,33536,33537,33538,33539,33540,33541,33542,33543,33544,33545,33546,33547,...,46146,46147,46148,46149,46150,46151,46176,46177,46178,46179,46180,46181,46182,46183,46184,46185,46186,46187,46245,46246,46247,46248,46249,46250,46251,46252,46253,46254,46294,46295,41729,41730,41731,41732,41733,41734,41735,41736,41737,41738
ID,33508,33509,33510,33511,33512,33513,33514,33515,33516,33517,33518,33519,33520,33521,33522,33523,33524,33525,33526,33527,33528,33529,33530,33531,33532,33533,33534,33535,33536,33537,33538,33539,33540,33541,33542,33543,33544,33545,33546,33547,...,46146,46147,46148,46149,46150,46151,46176,46177,46178,46179,46180,46181,46182,46183,46184,46185,46186,46187,46245,46246,46247,46248,46249,46250,46251,46252,46253,46254,46294,46295,41729,41730,41731,41732,41733,41734,41735,41736,41737,41738
Title,Gymnopedie for piano No. 1,"Gnossienne, for piano No. 1",Veiled Autumn,"Les Heures Persanes, 16 pieces for piano, Op. ...","Gnossienne, for piano No. 3","Gnossienne, for piano No. 5","Ananda, sonata for piano, Op. 303: 4. Vision o...","Les Heures Persanes, 16 pieces for piano, Op. ...",Intermezzo,"Mystic Flute, for piano, Op. 22","Les Heures Persanes, 16 pieces for piano, Op. ...",Gymnopedie for piano No. 2,"Wasserklavier, for 1 or 2 pianos","Les Heures Persanes, 16 pieces for piano, Op. ...",Petits chorals (12): No. 10,"Ananda, sonata for piano, Op. 303: 1. Andante",Gymnopedie for piano No. 3,"Les Heures Persanes, 16 pieces for piano, Op. ...",Apparent Heavenly Movement,Für Alina,Nahawand Taqsim,Nesma,Req' Samai' Taqsim,Haneen,Tahmila Sika Hozam,Bedouin Detuned Req',Visions,Samai' Nahawand Tawrados,Iqa'at Req',Alamein,Dahab,Devandra Banhart / I Feel Just Like A Child,Kings Of Convenience / I Don't Know What I Can...,Feist / Gatekeeper,José González / Slow Moves,Emiliana Torrini / Heartstopper,The Brian Jonestown Massacre / Time Is Honey (...,Dirty Tree & Cat Power / Great Waves,Anthony And The Johnsons / Hope There's Someone,Sufjan Stevens / John Wayne Gacy Jr.,...,The Strangest Thing,To Be Forgiven,Move On,Star People,You Have Been Loved,Free,Trompetenkonzert Es-dur Hob.VIIe:1 - 1. Allegro,Cellokonzert D-dur Hob.VIIb:1 - 1. Moderato,Cellokonzert D-dur Hob.VIIb:1 - 2. Adagio,Cellokonzert D-dur Hob.VIIb:1 - 3. Finale. All...,Trompetenkonzert Es-dur Hob.VIIe:1 - 2. Andante,Trompetenkonzert Es-dur Hob.VIIe:1 - 3. Allegro,Hornkonzert D-dur Hob.VIIb:3 - 1. Allegro,Hornkonzert D-dur Hob.VIIb:3 - 2. Adagio,Hornkonzert D-dur Hob.VIIb:3 - 3. Allegro,Hornkonzert D-dur Hob.VIId:4 - 1. Allegro mode...,Hornkonzert D-dur Hob.VIId:4 - 2. Adagio,Hornkonzert D-dur Hob.VIId:4 - 3. Allegro,1,10,2,3,4,5,6,7,8,9,Umi Says,Track 13,1,10,2,3,4,5,6,7,8,9
ArtistName,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,Tamara-Anna Cislowski,"Tawrados, Joseph & James","Tawrados, Joseph & James","Tawrados, Joseph & James","Tawrados, Joseph & James","Tawrados, Joseph & James","Tawrados, Joseph & James","Tawrados, Joseph & James","Tawrados, Joseph & James","Tawrados, Joseph & James","Tawrados, Joseph & James","Tawrados, Joseph & James",Various,Various,Various,Various,Various,Various,Various,Various,Various,...,George Michael,George Michael,George Michael,George Michael,George Michael,George Michael,Hakan Hardenberger,Heinrich Schiff,Heinrich Schiff,Heinrich Schiff,Hakan Hardenberger,Hakan Hardenberger,Hermann Baumann,Hermann Baumann,Hermann Baumann,Hermann Baumann,Hermann Baumann,Hermann Baumann,Blanck Mass,Blanck Mass,Blanck Mass,Blanck M

In [54]:
from OsOps import Ops

import pywintypes

pywintypes.datetime = pywintypes.TimeType

import os

In [55]:
Ops().storePKL( df, "mm_lib_df", os.getcwd() )

'c:\\Users\\KOJIN\\RSTR\\MonkeyPie\\mm_lib_df_230317_120649914451.pkl'

In [56]:
unpkl = Ops().unPklData( os.getcwd(), "mm_lib_df" )

In [57]:
unpkl.keys()

dict_keys(['mm_lib_df_230317_120649914451.pkl'])

In [58]:
unpkl_lib = unpkl['mm_lib_df_230317_120649914451.pkl']

In [60]:
unpkl_lib.shape

(25, 44103)

In [134]:
# add to now playing
SDB = win32com.client.Dispatch('SongsDB.SDBApplication')
SDB.ShutdownAfterDisconnect = False
SDB.Player.PlaylistAddTrack( flacs[0] )
SDB = None